In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

data_files = Path.home() / "Downloads" / "bike-rental-starter-kit" / "data"

print("Exists?", data_files.exists())
if data_files.exists():
    print("These are the files inside the data folder:")
    for p in data_files.iterdir():
        print(" -", p.name)

Exists? True
These are the files inside the data folder:
 - JC-201602-citibike-tripdata.csv
 - JC-201608-citibike-tripdata.csv
 - newark_airport_2016.csv
 - JC-201606-citibike-tripdata.csv
 - JC-201610-citibike-tripdata.csv
 - JC-201604-citibike-tripdata.csv
 - JC-201612-citibike-tripdata.csv
 - JC-201603-citibike-tripdata.csv
 - JC-201609-citibike-tripdata.csv
 - JC-201601-citibike-tripdata.csv
 - JC-201611-citibike-tripdata.csv
 - JC-201607-citibike-tripdata.csv
 - JC-201605-citibike-tripdata.csv


In [29]:
# inspecting the data files

citibike_files = sorted(data_files.glob("JC-2016*-citibike-tripdata.csv"))
weather_file = data_files / "newark_airport_2016.csv"

print("There are ",len(citibike_files), "citi bike files:",)
print("There is only one weather file in the data folder")

There are  12 citi bike files:
There is only one weather file in the data folder


In [26]:
cb_sample = pd.read_csv(citibike_files[0])

print("Looking at the first city bike file,", citibike_files[0].name)
print("The shape of the file is:", cb_sample.shape)
display(cb_sample.head(2))

print("These are the columns present in the city bike files:")
print(cb_sample.columns)

print("Checking for any missing data:")
print(cb_sample.isna().mean().sort_values(ascending=False).head(10))


Looking at the first city bike file, JC-201601-citibike-tripdata.csv
The shape of the file is: (7479, 15)


,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1


These are the columns present in the city bike files:
Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')
Checking for any missing data:
Birth Year                 0.04038
Trip Duration              0.00000
Start Time                 0.00000
Stop Time                  0.00000
Start Station ID           0.00000
Start Station Name         0.00000
Start Station Latitude     0.00000
Start Station Longitude    0.00000
End Station ID             0.00000
End Station Name           0.00000
dtype: float64


In [28]:
w_sample = pd.read_csv(weather_file)

print("Checking out the weather file:", weather_file.name)
print("This is the shape of the weather file:", w_sample.shape)
display(w_sample.head(2))

print("These are the columns present in the weather file:")
print(w_sample.columns)

print("Checking for any missing data:")
print(w_sample.isna().mean().sort_values(ascending=False).head(10))


Checking out the weather file: newark_airport_2016.csv
This is the shape of the weather file: (366, 16)


,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1


These are the columns present in the weather file:
Index(['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD',
       'TAVG', 'TMAX', 'TMIN', 'TSUN', 'WDF2', 'WDF5', 'WSF2', 'WSF5'],
      dtype='object')
Checking for any missing data:
PGTM       1.000000
TSUN       1.000000
WDF5       0.005464
WSF5       0.005464
STATION    0.000000
NAME       0.000000
DATE       0.000000
AWND       0.000000
PRCP       0.000000
SNOW       0.000000
dtype: float64


In [32]:
# cleaning the data

def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    return df

cb_sample = clean_columns(cb_sample)

print("These are the cleaned column titles for the citi bike files: ",cb_sample.columns)


These are the cleaned column titles for the citi bike files:  Index(['trip_duration', 'start_time', 'stop_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'birth_year', 'gender'],
      dtype='object')


In [45]:
cleaned_trips = []

for f in citibike_files:
    df = pd.read_csv(f)
    df = clean_columns(df)
    
    df["start_time"] = pd.to_datetime(df["start_time"])
    df["stop_time"] = pd.to_datetime(df["stop_time"])
    
    df["trip_date"] = df["start_time"].dt.date
    df["start_hour"] = df["start_time"].dt.hour
    df["trip_duration_minutes"] = df["trip_duration"] / 60
    
    df["gender"] = df["gender"].map(gender_map)
    df["birth_year"] = pd.to_numeric(df["birth_year"])
    df["age"] = 2016 - df["birth_year"]
    df.loc[(df["age"] < 5) | (df["age"] > 100), "age"] = None
    
    df = df[
        (df["trip_duration_minutes"] > 0) &
        (df["trip_duration_minutes"] <= 1440)
    ]
    
    cleaned_trips.append(df)

trips_df = pd.concat(cleaned_trips)

print("Trips dataframe shape:", trips_df.shape)
trips_df.head(2)


Trips dataframe shape: (247491, 19)


,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,trip_date,start_hour,trip_duration_minutes,age
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,Female,2016-01-01,0,6.033333,52.0
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,Male,2016-01-01,0,3.333333,54.0


In [48]:
weather_df = pd.read_csv(weather_file)

weather_df = clean_columns(weather_df)

weather_df["date"] = pd.to_datetime(weather_df["date"]).dt.date

weather_df = weather_df[["date", "tavg", "tmax", "tmin", "prcp", "snow", "awnd"]]

weather_df.rename(columns={
    "date": "weather_date",
    "tavg": "avg_temp",
    "tmax": "max_temp",
    "tmin": "min_temp",
    "prcp": "precipitation",
    "snow": "snowfall",
    "awnd": "avg_wind"
}, inplace=True)

weather_df.head(2)


,weather_date,avg_temp,max_temp,min_temp,precipitation,snowfall,avg_wind
0,2016-01-01,41,43,34,0.0,0.0,12.75
1,2016-01-02,36,42,30,0.0,0.0,9.40


In [50]:
OUTPUT_DIR = data_files / "cleaned"
OUTPUT_DIR.mkdir(exist_ok=True)

trips_df.to_csv(OUTPUT_DIR / "citibike_trips_clean.csv", index=False)
weather_df.to_csv(OUTPUT_DIR / "weather_daily_clean.csv", index=False)

print("Saved files:")
print(" -", OUTPUT_DIR / "citibike_trips_clean.csv")
print(" -", OUTPUT_DIR / "weather_daily_clean.csv")


Saved files:
 - /Users/cheikh/Downloads/bike-rental-starter-kit/data/cleaned/citibike_trips_clean.csv
 - /Users/cheikh/Downloads/bike-rental-starter-kit/data/cleaned/weather_daily_clean.csv
